In [2]:
import torch

# 機械学習で出る内積計算の例

In [2]:
# 1次元 & 1次元
a   = torch.randn([4])
b   = torch.randn([4])
c   = torch.dot(a, b)
print('1次元 & 1次元')
print(c)
print('------')

# 2次元 & 2次元
a   = torch.randn([3, 4])
b   = torch.randn([4, 9])
c   = torch.mm(a, b)
print('2次元 & 2次元')
print(c.shape)
print('------')

# 2次元 & 1次元
a   = torch.randn([3, 4])
b   = torch.randn([4])
c   = torch.mv(a, b)
print('2次元 & 1次元')
print(c, c.shape)
print('------')

# 3次元 & 3次元 (batch 入り2次元と考える)
# 2次元 & 1次元
a   = torch.randn([10, 3, 4])
b   = torch.randn([10, 4, 9])
c   = torch.bmm(a, b)
print('3次元 & 3次元')
print(c.shape)
print('------')

# 一般: matmul 使用
## 1次元 & 1次元
a   = torch.randn([4])
b   = torch.randn([4])
c   = torch.matmul(a, b)
print(c)

## 2次元 & 1次元
a   = torch.randn([3, 4])
b   = torch.randn([4])
c   = torch.matmul(a, b)
print(c.shape)

## 3次元 & 3次元
a   = torch.randn([10, 3, 4])
b   = torch.randn([10, 4, 9])
c   = torch.matmul(a, b)
print('3次元 & 3次元: ', c.shape)

## 3次元 & 2次元
a   = torch.randn([10, 3, 4])
b   = torch.randn([4, 5])
c   = torch.matmul(a, b)
print('3次元 & 2次元: ', c.shape)

##  4次元 & 3次元
a   = torch.randn([10, 2, 3, 4])
b   = torch.randn([2, 4, 5])
c   = torch.matmul(a, b)
print(c.shape)


1次元 & 1次元
tensor(-0.2639)
------
2次元 & 2次元
torch.Size([3, 9])
------
2次元 & 1次元
tensor([-2.3500, -3.8287,  2.5689]) torch.Size([3])
------
3次元 & 3次元
torch.Size([10, 3, 9])
------
tensor(-2.1956)
torch.Size([3])
3次元 & 3次元:  torch.Size([10, 3, 9])
3次元 & 2次元:  torch.Size([10, 3, 5])
torch.Size([10, 2, 3, 5])


# アダマール積の例

### 目標
Jin-Hwa et al. (2018) "Bilinear Attention Networks" の (10)式のlogits項
$$
    A_{i, j} = ((\mathbb{1}\cdot{\boldsymbol p}^{T})\circ{\boldsymbol X}^{T}{\boldsymbol U}){\boldsymbol V}^{T}{\boldsymbol Y}
$$
を計算したい。ここで、$\mathbb{1} \in \mathbb{R}^{\rho}, {\boldsymbol p} \in \mathbb{R}^{K},  {\boldsymbol X} \in \mathbb{R}^{N\times \rho},  {\boldsymbol Y} \in \mathbb{R}^{M\times \phi}, {\boldsymbol U} \in \mathbb{R}^{N\times K}, {\boldsymbol V} \in \mathbb{R}^{M\times K}$であるので、結局 $${\boldsymbol A} \in \mathbb{R}^{\rho \times \phi} $$





In [10]:
# 2次元 x 2次元のアダマール積
a = torch.tensor([[1,2],[3,4]])
b = torch.tensor([[2,3],[4,5]])
c1 = a*b   # * による演算
print('c1=', c1)
c2 = torch.einsum('ij,ij->ij', a, b)   # einsum を使う
print('c2=', c2)

# 3次元 x 3次元で最初の1次元がバッチの次元で残りがアダマール積
a = torch.tensor([[[1, 2, 3, 4], [4, 3, 2, 1], [2, 3, 4, 5]], [[2, 3, 4 ,5], [5, 4 , 3, 2], [3, 4, 5, 6]]])
b = torch.tensor([[[1, 2, 3, 4], [4, 3, 2, 1], [2, 3, 4, 5]], [[2, 3, 4 ,5], [5, 4 , 3, 2], [3, 4, 5, 6]]])
c = torch.einsum('ijk,ijk->ijk', a, b)
print('a=', a)
print('b=', b)
print('c=', c)
print(c.shape)
 
# (9)式で batch ありを考える。X'=U^{T}X, Y'=V^{T}Y がすでに計算済みとする。X': (b_size, rho, K), Y': (b_size, K, phi) の次元
# batch_size: 2次元, K: 3次元, rho: : 4, phi: 5 と仮定すると
uni = torch.Tensor([[1, 1, 1, 1], [1, 1, 1, 1]])
p   = torch.Tensor([[1, 2, 3], [2, 3, 4]])
x_ = torch.Tensor([[[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6]], [[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60]]])
y_ = torch.Tensor([[[1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7]], [[.1, .2, .3, .4, .5], [.2, .3, .4, .5, .6], [.3, .4, .5, .6, .7]]])

p_ = torch.einsum('bi,bj->bij', uni, p)
print(p_.shape)
print(p_)

A = torch.einsum('brk,brk,bkp->brp', p_, x_, y_)
print('A_size: ', A.shape)
print('A: ', A)

# p_ を計算しなくても、einsumの機能で p_ の計算をスキップできる
p__ = p.unsqueeze(0)
A2 = torch.einsum('xrk,brk,bkp->brp', p_, x_, y_)
print('A2_size: ', A.shape)
print('A2: ', A)


# A = torch.einsum('', uni, p, )
# さらに、glimpse の次元まで考えると...



c1= tensor([[ 2,  6],
        [12, 20]])
c2= tensor([[ 2,  6],
        [12, 20]])
a= tensor([[[1, 2, 3, 4],
         [4, 3, 2, 1],
         [2, 3, 4, 5]],

        [[2, 3, 4, 5],
         [5, 4, 3, 2],
         [3, 4, 5, 6]]])
b= tensor([[[1, 2, 3, 4],
         [4, 3, 2, 1],
         [2, 3, 4, 5]],

        [[2, 3, 4, 5],
         [5, 4, 3, 2],
         [3, 4, 5, 6]]])
c= tensor([[[ 1,  4,  9, 16],
         [16,  9,  4,  1],
         [ 4,  9, 16, 25]],

        [[ 4,  9, 16, 25],
         [25, 16,  9,  4],
         [ 9, 16, 25, 36]]])
torch.Size([2, 3, 4])
torch.Size([2, 4, 3])
tensor([[[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[2., 3., 4.],
         [2., 3., 4.],
         [2., 3., 4.],
         [2., 3., 4.]]])
A_size:  torch.Size([2, 4, 5])
A:  tensor([[[ 36.,  50.,  64.,  78.,  92.],
         [ 50.,  70.,  90., 110., 130.],
         [ 64.,  90., 116., 142., 168.],
         [ 78., 110., 142., 174., 206.]],

        [[ 50.,  70.,  90.

In [24]:
a = torch.Tensor([1,1,1])
b = torch.Tensor([1,2,3,4])
c = torch.einsum('i,j->ij', a, b)  # ある意味broadcast
print(c, c.shape)

c[0, :]

tensor([[1., 2., 3., 4.],
        [1., 2., 3., 4.],
        [1., 2., 3., 4.]]) torch.Size([3, 4])


tensor([1., 2., 3., 4.])

In [12]:
x = torch.Tensor([[[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6]], [[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60]]])
torch.sum(x, 2).shape

print(torch.sum(x, 2))
print(x.sum(2))

tensor([[  6.,   9.,  12.,  15.],
        [ 60.,  90., 120., 150.]])
tensor([[  6.,   9.,  12.,  15.],
        [ 60.,  90., 120., 150.]])
